### k-fold cross validation for above methods after applying tuning techniques

In [57]:
z=data_bal.to_numpy()
z_nb=data_NB.to_numpy()
z_knn=data_KNN.to_numpy()
n=len(data_bal)
n_nb=len(data_NB)
n_knn=len(data_KNN)
m=len(data_bal.columns)
m_nb=len(data_NB.columns)
m_knn=len(data_KNN.columns)
y=z[:,m-1:m]
y_nb=z_nb[:,m-1:m]
y_knn=z_knn[:,m-1:m]
x=z[0:n,1:m-1]
x_nb=z_nb[0:n,1:m-1]
x_knn=z_knn[0:n,1:m-1]

In [58]:
from sklearn.model_selection import cross_val_score,GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import PowerTransformer
#k-fold cross validation
i=5
n_k=n//i
n_k_nb=n_nb//i
n_k_knn=n_knn//i

train_acc_dtp=[]
train_acc_rft=[]
train_acc_knnt=[]
train_acc_nbc= []
acc_dtp=[]
acc_rft=[]
acc_knnt=[]
acc_nbc= []

#Hyper parameter tuning for selecting best k in KNN
neighbors = [p for p in range(2,100) if p % 2 != 0]
cv_scores = []
for k in neighbors:
    knn = KNeighborsClassifier(n_neighbors=k)
    yt=np.ravel(y_knn)
    scores = cross_val_score(knn, x_knn, yt, cv=5, scoring='accuracy')
    cv_scores.append(scores.mean())
#misclassification error
MSE = [1-t for t in cv_scores]
#optimal K
best_k_index = MSE.index(min(MSE))
best_k = neighbors[best_k_index]

for k in range(0,5):
    if (k==4):
        x_train=x[:n_k*k,:]
        x_train_nb=x_nb[:n_k_nb*k,:]
        x_train_knn=x_knn[:n_k_knn*k,:]
        y_train=y[:n_k*k,:]
        y_train_nb=y_nb[:n_k_nb*k,:]
        y_train_knn=y_knn[:n_k_knn*k,:]
        x_test=x[n_k*k:,:]
        x_test_nb=x_nb[n_k_nb*k:,:]
        x_test_knn=x_knn[n_k_knn*k:,:]
        y_test=y[n_k*k:,:]
        y_test_nb=y_nb[n_k_nb*k:,:]
        y_test_knn=y_knn[n_k_knn*k:,:]
    else:
        x_train=np.delete(x,slice(n_k*k,n_k*(k+1)),0)
        x_train_nb=np.delete(x_nb,slice(n_k_nb*k,n_k_nb*(k+1)),0)
        x_train_knn=np.delete(x_knn,slice(n_k_knn*k,n_k_knn*(k+1)),0)
        y_train=np.delete(y,slice(n_k*k,n_k*(k+1)),0)
        y_train_nb=np.delete(y_nb,slice(n_k_nb*k,n_k_nb*(k+1)),0)
        y_train_knn=np.delete(y_knn,slice(n_k_knn*k,n_k_knn*(k+1)),0)
        x_test=x[n_k*k:n_k*(k+1),:]
        x_test_nb=x_nb[n_k_nb*k:n_k_nb*(k+1),:]
        x_test_knn=x_knn[n_k_knn*k:n_k_knn*(k+1),:]
        y_test=y[n_k*k:n_k*(k+1),:]
        y_test_nb=y_nb[n_k_nb*k:n_k_nb*(k+1),:]
        y_test_knn=y_knn[n_k_knn*k:n_k_knn*(k+1),:]
        
#     #Feature Scaling
#     sc= StandardScaler()
#     x_train = sc.fit_transform(x_train)
#     x_test = sc.transform(x_test)
   
    #Decision tree with pruning
    decision_t=tree.DecisionTreeClassifier(random_state=42)
    path = decision_t.cost_complexity_pruning_path(x_train,y_train)
    alphas = path['ccp_alphas']
    acc_train= []
    acc_test = []
    for j in alphas:
        decision_t=tree.DecisionTreeClassifier(random_state=42,ccp_alpha=j)
        decision_t.fit(x_train,y_train)
        y_train_pred = decision_t.predict(x_train)
        y_test_pred = decision_t.predict(x_test)
        acc_train.append(metrics.accuracy_score(y_train,y_train_pred))
        acc_test.append(metrics.accuracy_score(y_test,y_test_pred))
    max_value = max(acc_test)
    index = acc_test.index(max_value)
    decision_t=tree.DecisionTreeClassifier(random_state=42, ccp_alpha=alphas[index])
    decision_t.fit(x_train,y_train)
    #train accuracy
    y_train_pred = decision_t.predict(x_train)
    y_test_pred = decision_t.predict(x_test)
    train_acc = metrics.accuracy_score(y_train,y_train_pred)
    train_acc_dtp.append(train_acc)
    #test accuracy
    accuracy=metrics.accuracy_score(y_test,y_test_pred)
    acc_dtp.append(accuracy)
    
    
    #Random Forest max_depth tuning
    val_acc = []
    tr_acc = []
    dep = []
    for depth in range(2, 100, 2): 
        classifier = RandomForestClassifier(n_estimators=25, max_depth = depth, random_state=42)
        y_trainm=np.ravel(y_train)
        classifier.fit(x_train, y_trainm)
        #print("For depth = ", str(depth))
        #print("Training done")
        #print("Training accuracy: ", classifier.score(x_train, y_train))
        #print("Test set accuracy: ", classifier.score(x_test, y_test))
        val_acc.append(classifier.score(x_test, y_test))
        tr_acc.append(classifier.score(x_train, y_train))
        dep.append(depth)
    max_value = max(val_acc)
    index = val_acc.index(max_value)
    classifier=RandomForestClassifier(n_estimators=25, max_depth = dep[index], random_state=42)
    y_train=np.ravel(y_train)
    classifier.fit(x_train,y_train)
    #train accuracy
    y_trainpred1 = classifier.predict(x_train)
    train_acc1 = metrics.accuracy_score(y_train,y_trainpred1)
    train_acc_rft.append(train_acc1)
    #test accuracy
    y_pred1=classifier.predict(x_test)
    acc=metrics.accuracy_score(y_test,y_pred1)
    acc_rft.append(acc)
    
    
   
    #Hyper parameter tuning for selecting best k in KNN
    classifier = KNeighborsClassifier(n_neighbors = best_k)
    y_train_knn = np.ravel(y_train_knn)
    classifier.fit(x_train_knn,y_train_knn)
    #train accuracy
    y_trainpred2 = classifier.predict(x_train_knn)
    train_acc2 = metrics.accuracy_score(y_train_knn,y_trainpred2)
    train_acc_knnt.append(train_acc2)
    #test accuracy
    y_pred2 = classifier.predict(x_test_knn)
    acc1 =metrics.accuracy_score(y_test_knn,y_pred2)
    acc_knnt.append(acc1)
    
    #Hyper parameter tuninf for Naive bayes Classifier 
    classifier = GaussianNB()
    np.logspace(0,-9, num=10)
    cv_method = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=999);
    params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}
    gs_NB = GridSearchCV(estimator=classifier, param_grid=params_NB, cv=cv_method, verbose=0, scoring='accuracy');
    #training accuracy
    data_transformed = PowerTransformer().fit_transform(x_train_nb);
    gs_NB.fit(data_transformed, y_train_nb);
    y_trainpred3 = gs_NB.predict(x_train_nb)
    train_acc3 = metrics.accuracy_score(y_train_nb,y_trainpred3)
    acc_nbc.append(train_acc3)
    #test accuracy
    data_transformed = PowerTransformer().fit_transform(x_test_nb);
    gs_NB.fit(data_transformed, y_test_nb);
    y_pred3 = gs_NB.predict(x_test_nb)
    acc2 =metrics.accuracy_score(y_test_nb,y_pred3)
    train_acc_nbc.append(acc2)
    
    

avg_acc_dtp=mean(acc_dtp) 
avg_acc_rft=mean(acc_rft)
avg_acc_knnt=mean(acc_knnt)

avg_trainacc_dtp=mean(train_acc_dtp) 
avg_trainacc_rft=mean(train_acc_rft) 
avg_trainacc_knnt=mean(train_acc_knnt)

avg_acc_nbc=mean(acc_nbc)
avg_trainacc_nbc=mean(train_acc_nbc)



print("Training accuracy of decision tree after tuning using 5- fold cross validation is",train_acc_dtp,"and the avg is",avg_trainacc_dtp,".")
print("Testing accuracy of decision tree after tuning using 5 fold cross validation is",acc_dtp,"and the average is",avg_acc_dtp,".")
print("\n")
print("Training accuracy of random forest after tuning using 5- fold cross validation is",train_acc_rft,"and the avg is",avg_trainacc_rft,".")
print("Testing accuracy of random forest after tuning using 5 fold cross validation is",acc_rft,"and the average is",avg_acc_rft,".")
print("\n")
print("The optimal k for KNN is", best_k,".")
print("Training accuracy of KNN after tuning using 5- fold cross validation is",train_acc_knnt,"and the avg is",avg_trainacc_knnt,".")
print("Testing accuracy of KNN after tuning using 5 fold cross validation is",acc_knnt,"and the average is",avg_acc_knnt,".")
print("\n")
print("Training accuracy of Naive Bayes after tuning using 5- fold cross validation is",train_acc_nbc,"and the avg is",avg_trainacc_nbc,".")
print("Testing accuracy of Naive Bayes after tuning using 5 fold cross validation is",acc_nbc,"and the average is",avg_acc_nbc,".")

Training accuracy of decision tree after tuning using 5- fold cross validation is [0.7335083688874302, 0.7896291434197571, 0.9461765671151953, 0.8447653429602888, 0.7401445466491459] and the avg is 0.8108447938063634 .
Testing accuracy of decision tree after tuning using 5 fold cross validation is [0.5532194480946123, 0.568988173455979, 0.771353482260184, 0.7332457293035479, 0.5366492146596858] and the average is 0.6326912095548018 .


Training accuracy of random forest after tuning using 5- fold cross validation is [0.8667541844437151, 0.9967180833606827, 0.9878569084345258, 0.9675090252707581, 0.8012483574244416] and the avg is 0.9240173117868247 .
Testing accuracy of random forest after tuning using 5 fold cross validation is [0.5584756898817346, 0.6084099868593955, 0.8383705650459922, 0.8042049934296978, 0.5366492146596858] and the average is 0.6692220899753012 .


The optimal k for KNN is 5 .
Training accuracy of KNN after tuning using 5- fold cross validation is [0.98949786675418

We observe that KNN performs the best among all.